## QCLab: 3-Qubit Quantum Born Machine Trained on Target Probabilities

A **Quantum Born Machine (QBM)** is a generative quantum machine learning model that learns probability distributions using the Born rule. The QBM is given a target distribution and must adjust its quantum circuit parameters so that the distribution of its measurement outcomes becomes similar to the given one. This lab is an expansion of the "Minimal 2-qubit QBM" lab to three qubits. The simple naive minimization used earlier is now replaced with the COBYLA optimizer from the SciPy library. This setup is closer to a real situation, where only the target probabilities are given, without access to the required results.
In this particular example, the probability distribution represents the number of customers visiting a small coffee shop each hour over an 8-hour day. The goal is to simulate this traffic pattern using a quantum Born machine.  

![3-qubit-QBM](images/3-qubit-QBM.png)

In this particular example, the probability distribution represents the number of customers visiting a small coffee shop each hour over an 8-hour day, given as 
$p_{\text{dist}} = [0.25, 0.18, 0.09, 0.04, 0.04, 0.09, 0.18, 0.25]$
The goal is to simulate this traffic pattern using a quantum Born machine. To be able to achieve this, the given distribution is assigned to the 3-bit strings in binary order corresponding the given probability.

---

### Sampling, Model Distribution and Optimization

The model distribution $p_{\text{model}}$ is obtained by running the circuit multiple times (`sample_qbm(theta, shots=1000)`) on a simulator and normalizing counts:

$$p_{\text{model}}(x) = \frac{\text{counts}(x)}{\sum_{y}\text{counts}(y)}$$  

where $x$ is a bitstring representing measurement outcomes. 

To compare the learned distribution with the target, we use the L2 loss (squared error), defined as:

$$L_2(\theta) = \sum_{x \in \{0,1\}^3} \big( p_{\text{target}}(x) - p_{\text{model}}(x) \big)^2$$  

Optimization proceeds as follows:  
- Start with random parameters `theta0`.  
- Use the COBYLA optimizer from SciPy to minimize the loss $L(\theta)$.  
- Obtain the optimized parameters `theta_opt`.  


### Task

- Implement a 3-qubit Quantum Born Machine (QBM) that learns a given target probability distribution.  
- Construct a parameterized circuit using $R_y(\theta)$ rotations and entangling CNOT gates.  
- Train the circuit with the COBYLA optimizer from SciPy using only the target probabilities.  
- Simulate the resulting circuit to approximate the target distribution representing customer visits to a coffee shop over 8 hours.  

---

### Expected Output

- A plot of the QBM circuit with optimized rotation angles.  
- Overlayed bar plots comparing the target distribution $p_{\text{target}}$ and the model distribution $p_{\text{model}}$ for a visual check of fit.  
- Printed results of the optimization, including the optimized parameters `theta_opt`.  

---

### Experimentation

- Try different initial values of `theta0` to observe how the optimizer converges.  
- Compare the effects of using L1 vs L2 loss functions on the final learned distribution.  
- Vary the number of shots in `sample_qbm` to see how sampling noise influences the accuracy of the model distribution.  
- Extend the circuit by adding more entangling gates or depth to explore how expressivity affects learning.  


In [ ]:
# ====================================================
# QCLab: 3-Qubit Quantum Born Machine 
#        Trained on Target Probabilities
# <QC|CT> qcict.org
# ====================================================

from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_distribution
from qiskit.visualization import circuit_drawer
from scipy.optimize import minimize
import numpy as np

# -- given target distribution --
p_dist = [0.25, 0.18, 0.09, 0.04, 0.04, 0.09, 0.18, 0.25]

# -- assign probabilities to 3-bit strings --
p_target = {format(i, '03b'): p for i, p in enumerate(p_dist)}

# -- create a QBM circuit with 3 parameters --
def create_qbm_circuit(theta):
    qc = QuantumCircuit(3)
    for i in range(3):
        qc.ry(theta[i], i)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.measure_all()
    return qc

# -- sample from the QBM circuit --
def sample_qbm(theta, shots=1000):
    qc = create_qbm_circuit(theta)
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    
    # normalize counts to get model distribution
    total = sum(counts.values())
    p_model = {}
    for bitstring, count in counts.items():
        p_model[bitstring] = count / total
    return p_model

# -- define L2 loss function for optimizer --
def l2_loss(theta):
    p_model = sample_qbm(theta)
    return sum((p_target[k] - p_model.get(k, 0)) ** 2 for k in p_target)

# -----------------------------------------------
#                main program
# -----------------------------------------------

# -- run optimization loop --
theta0 = np.random.uniform(0, 2 * np.pi, 6)
result = minimize(l2_loss, theta0, method='COBYLA')
theta_opt = result.x

# -- sample final QBM output --
p_qbm = sample_qbm(theta_opt)

# -- plot the circuit with optimized angles --
qc = create_qbm_circuit(theta_opt)
display(circuit_drawer(qc, style="bw", output="mpl"))

# map bitstrings -> hour labels
hour = {
    '000': 'Hour 1', '001': 'Hour 2', '010': 'Hour 3', '011': 'Hour 4',
    '100': 'Hour 5', '101': 'Hour 6', '110': 'Hour 7', '111': 'Hour 8'
}

display(plot_distribution(
    [{hour.get(k, k): v for k, v in d.items()} for d in [p_target, p_qbm]], 
    legend=["Target", "Model"], title="Coffee Shop Traffic", bar_labels=True))